# Test Validity Inverse Wishart binned

In [1]:
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
import jax
from jax import random, dtypes
import jax.numpy as jnp
import jax.scipy as jsp
import jax_healpy as jhp
import scipy
import healpy as hp
import astropy.io.fits as fits
import camb
import numpyro

import micmac as micmac

In [2]:
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('')))+'/tutorials/')

In [ ]:
# working_directory_path = os.path.abspath('') + '/'
working_directory_path = '/Users/mag/Documents/PHD1Y/Space_Work/Pixel_non_P2D/MICMAC/test_playground/validation_chain_spatialvariability_v4/'
directory_save_file = working_directory_path + 'save_directory/'
directory_toml_file = working_directory_path + 'main_params/'
directory_additional_params = working_directory_path + 'additional_params/'

path_toml_file = directory_toml_file + 'corr_fullsky_SO_v1a.toml'
path_additional_params = directory_additional_params + 'add_corr_fullsky_SO_d0s0_nside1_v1a.toml'

In [ ]:
MICMAC_obj = micmac.create_MICMAC_sampler_from_toml_file(path_toml_file)

In [ ]:
with open(path_additional_params) as f:
    dictionary_additional_parameters = toml.load(f)
f.close()

## Few options to activate/deactivte

In [ ]:
use_mask = False
foreground_bool = False

In [ ]:
instrument = fgbuster.get_instrument(dictionary_additional_parameters['instrument_name'])

np.random.seed(dictionary_additional_parameters['seed_realization_input'])
noise_map = get_noise_realization(MICMAC_obj.nside, instrument)[:, 1:, :]

if foreground_bool:
    fgs_model = dictionary_additional_parameters['fgs_model']
    np.random.seed(dictionary_additional_parameters['seed_realization_input'])
    freq_maps_fgs_denoised = get_observation(instrument, fgs_model_, nside=MICMAC_obj.nside, noise=False)[:, 1:, :]   # keep only Q and U
    

    freq_maps_fgs = freq_maps_fgs_denoised + noise_map
else:
    freq_maps_fgs = noise_map

In [ ]:
if use_mask:
    path_mask = '/Users/mag/Documents/PHD1Y/Masks/mask_SAT_apodized.fits'
    mask = hp.read_map(path_mask, verbose=False)
    MICMAC_obj.mask = mask
else:
    MICMAC_obj.mask = np.ones(MICMAC_obj.n_pix)

In [ ]:
np.random.seed(seed_realization_input+1)
input_freq_maps, input_cmb_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)
input_freq_maps_masked = input_freq_maps*MICMAC_obj.mask

theoretical_r0_total = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)#[partial_indices_polar,:]
theoretical_r1_tensor = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)#[partial_indices_polar,:]


In [ ]:
init_mixing_matrix_obj = micmac.InitMixingMatrix(freqs=MICMAC_obj.frequency_array, 
                                                 ncomp=MICMAC_obj.n_components, 
                                                 pos_special_freqs=MICMAC_obj.pos_special_freqs, 
                                                 spv_nodes_b=MICMAC_obj.spv_nodes_b)
exact_params_mixing_matrix = init_mixing_matrix_obj.init_params()

MICMAC_obj.update_params(exact_params_mixing_matrix)
mixing_matrix_sampled = MICMAC_obj.get_B(jax_use=True)

invBtinvNB = micmac.get_inv_BtinvNB(MICMAC_obj.freq_inverse_noise, mixing_matrix_sampled)
BtinvN_sqrt = micmac.get_BtinvN(jnp.sqrt(MICMAC_obj.freq_inverse_noise), mixing_matrix_sampled)

In [ ]:
true_CMB_c_ell = np.zeros_like((MICMAC_obj.number_correlations, MICMAC_obj.lmax+1-MICMAC_obj.lmin))
true_CMB_c_ell[:MICMAC_obj.nstokes,:] = theoretical_r0_total + MICMAC_obj.r_true*theoretical_r1_tensor


In [ ]:
red_cov_matrix = micmac.get_reduced_matrix_from_c_ell(true_CMB_c_ell)
red_cov_matrix_sqrt = micmac.get_sqrt_reduced_matrix_from_matrix_jax(red_cov_matrix)

## Prepare CMB map

In [ ]:
true_cmb_map = input_cmb_maps[0,:]

In [ ]:
s_cML = micmac.get_Wd(MICMAC_obj.freq_inverse_noise, mixing_matrix_sampled, input_freq_maps, jax_use=False)[0, :, :]

In [ ]:
jax_key_PNRG = jax.random.PRNGKey(MICMAC_obj.seed + 2)

wiener_filter_map = MICMAC_obj.solve_generalized_wiener_filter_term_v2d(s_cML, red_cov_matrix_sqrt, invBtinvNB, initial_guess=jnp.empty(0),precond_func=None)

fluctuation_map = MICMAC_obj.get_fluctuating_term_maps_v2d(red_cov_matrix_sqrt, 
                                                           invBtinvNB, 
                                                           BtinvN_sqrt, 
                                                           jax_key_PNRG, 
                                                           map_random_realization_xi=None, 
                                                           map_random_realization_chi=None, 
                                                           initial_guess=jnp.empty(0),
                                                           precond_func=None)

s_c_sample = wiener_filter_map + fluctuation_map

In [ ]:
list_correl = ['EE', 'BB', 'EB']


ell_arange = np.arange(red_cov_approx_matrix.shape[0]) + MICMAC_obj.lmin


BtinvNB = micmac.get_inv_BtinvNB(freq_inverse_noise, mixing_matrix_sampled)
# BtinvN_sqrt = micmac.get_BtinvN(sp.linalg.sqrtm(freq_inverse_noise), mixing_matrix_sampled)
BtinvN_sqrt = micmac.get_BtinvN(np.sqrt(freq_inverse_noise), mixing_matrix_sampled)


_cl_noise_harm = micmac.get_Cl_noise(np.array(instrument['depth_p']), mixing_matrix_sampled.mean(axis=2), lmax)[0,0,lmin:]
red_cl_noise_harm = micmac.get_reduced_matrix_from_c_ell(np.array([_cl_noise_harm,_cl_noise_harm,np.zeros_like(_cl_noise_harm)]))
cl_noise_harm = micmac.get_c_ells_from_red_covariance_matrix(red_cl_noise_harm)

analytical_WF_harm = np.linalg.pinv(np.linalg.pinv(red_cov_matrix) + np.linalg.pinv(red_cl_noise_harm))
analytical_WF_map = micmac.maps_x_reduced_matrix_generalized_sqrt_sqrt(s_cML/BtinvNB[0,0]/hp.nside2resol(MICMAC_obj.nside)**2,
                                                                    analytical_WF_harm, lmin=lmin, n_iter=n_iter)


full_map_s_c = wiener_filter_map + fluctuation_map
full_map_s_cML_extended = np.vstack([np.zeros_like(s_cML[0,...]), s_cML])
c_ell_s_cML = hp.anafast(full_map_s_cML_extended, lmax=MICMAC_obj.lmax, iter=MICMAC_obj.n_iter)[indices_polar,MICMAC_obj.lmin:]

analytical_WF_map_ext = np.vstack([np.zeros_like(analytical_WF_map[0]),analytical_WF_map])
analytical_WF_cells = hp.anafast(analytical_WF_map_ext, lmax=MICMAC_obj.lmax, iter=MICMAC_obj.n_iter)[indices_polar,MICMAC_obj.lmin:]
analytical_fluct_c_ell = micmac.get_c_ells_from_red_covariance_matrix(analytical_WF_harm.copy())

s_c_WF_maps_extended = np.vstack([np.zeros_like(wiener_filter_map[0,...]), wiener_filter_map])
c_ells_s_c_WF = hp.anafast(s_c_WF_maps_extended, lmax=MICMAC_obj.lmax, iter=MICMAC_obj.n_iter)[indices_polar,MICMAC_obj.lmin:]

s_c_fluct_maps_extended = np.vstack([np.zeros_like(fluctuation_map[0,...]), fluctuation_map])
c_ells_s_c_fluct = hp.anafast(s_c_fluct_maps_extended, lmax=MICMAC_obj.lmax, iter=MICMAC_obj.n_iter)[indices_polar,MICMAC_obj.lmin:]

s_c_recovered = wiener_filter_map + fluctuation_map
s_c_recovered_ext = np.vstack([np.zeros_like(s_c_recovered[0]),s_c_recovered])
c_ells_s_c = hp.anafast(s_c_recovered_ext, lmax=MICMAC_obj.lmax, iter=MICMAC_obj.n_iter)[indices_polar,MICMAC_obj.lmin:]

plt.figure(figsize=(14,8))
plt.suptitle(r"Comparison $s_c$ CG vs theoretical for iteration {}".format(iteration))

for j in range(n_correl_to_plot):
    plt.subplot(331+3*j)
    plt.plot(np.arange(c_ells_s_c_WF.shape[-1])+MICMAC_obj.lmin, analytical_WF_cells[j,:], label='Analytical WF')
    plt.plot(np.arange(c_ells_s_c_WF.shape[-1])+MICMAC_obj.lmin, c_ells_s_c_WF[j,:], '-.', label='CG-WF')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, c_ell_s_cML[j,:], ':', label='$s_{c,ML}$')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, cl_noise_harm[j,:], 'r:', label='noise $N_c$')
    if j <= MICMAC_obj.nstokes-1:
        plt.loglog()
    else:
        plt.xscale('log')
        plt.yscale('symlog')
        plt.plot([MICMAC_obj.lmin,MICMAC_obj.lmax+1], [0,0], 'k:')
    plt.title('WF term - {}'.format(list_correl[j]))
    plt.ylabel("$C_\ell$ [$uK^2$]")
    plt.xlabel(r"$\ell$")
    plt.legend()

    plt.subplot(332+3*j)
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, analytical_fluct_c_ell[j,:], label='Analytical fluct')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, c_ells_s_c_fluct[j,:], '-.', label='CG-Fluct')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, cl_noise_harm[j,:], 'r:', label='noise $N_c$')

    if j <= MICMAC_obj.nstokes-1:
        plt.loglog()
    else:
        plt.xscale('log')
        plt.yscale('symlog')
        plt.plot([MICMAC_obj.lmin,MICMAC_obj.lmax+1], [0,0], 'k:')
    plt.title('Fluctuation term - {}'.format(list_correl[j]))
    plt.ylabel("$C_\ell$ [$uK^2$]")
    plt.xlabel(r"$\ell$")
    plt.legend()
    
    plt.subplot(333+3*j)
    plt.plot(np.arange(c_ells_s_c_WF.shape[-1])+MICMAC_obj.lmin, true_CMB_c_ell[j,:], label='True CMB')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, c_ells_s_c[j,:], '-.', label='$s_c$ sample')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, c_ell_s_cML[j,:], ':', label='$s_{c,ML}$')
    plt.plot(np.arange(c_ells_s_c_fluct.shape[-1])+MICMAC_obj.lmin, cl_noise_harm[j,:], 'r:', label='noise $N_c$')

    if j <= MICMAC_obj.nstokes-1:
        plt.loglog()
        plt.plot([MICMAC_obj.lmin,MICMAC_obj.lmax+1], [0,0], 'k:')
    else:
        plt.xscale('log')
        plt.yscale('symlog')
        plt.plot([MICMAC_obj.lmin,MICMAC_obj.lmax+1], [0,0], 'k:')
    plt.title('Full $s_c$ term - {}'.format(list_correl[j]))
    plt.ylabel("$C_\ell$ [$uK^2$]")

    plt.xlabel(r"$\ell$")
    plt.legend()
plt.show()

## Prepare binning

In [ ]:
delta_ell = 1

In [ ]:
nb_bin = (MICMAC_obj.lmax-MICMAC_obj.lmin+1)//delta_ell
MICMAC_obj.bin_ell_distribution = MICMAC_obj.lmin + jnp.arange(nb_bin+1)*delta_ell
MICMAC_obj.maximum_number_dof = int(MICMAC_obj.bin_ell_distribution[-1]**2 - MICMAC_obj.bin_ell_distribution[-2]**2)
